# Q-Learning en Machine Learning - Parte I

**SEGÚN FEDERICO:**

Es un tipo de algoritmo de aprendizaje por refuerzo que le permite a un agente imaginario aprender a tomar decisiones óptimas y a alcanzar un objetivo en ese entorno o ambiente determinado.

El agente opera **aprendiendo los valores que sean más convenientes para cada paso** que tenga que dar, y a esos pasos los implementamos en un par de datos que definen la acción que tiene que tomar y el estado en que queda.

Entonces, Q-Learning implica un agente que debe identificar qué pasos realizar y la **calidad de esos pasos** la vamos a denominar con la letra `Q`.

Los valores `Q`, es decir, los valores de sus pasos que se definen por los valores de la acción y del Estado, van a ayudar a este agente a decidir qué acción tiene que tomar en cada paso.

### Ecuación de Bellman para actualizar valores Q

![Bellman.png](./assets/Bellman.png)

Donde:

- `𝑄(𝑠,a)` = función 𝑄 que se está actualizando. Representa el valor actual estimado de tomar la acción 𝑎 en el estado 𝑠.

- `𝑠` = estado (state)

- `a` = acción (action)

- `α` = tasa de aprendizaje o **learning rate** (usualmente, un número entre 0 y 1). Controla qué tan rápido actualizamos la estimación del valor. Si α es cercano a 1, significa que confiamos mucho en las nuevas experiencias, y si es cercano a 0, confiamos más en los valores previos. Dicho de otra manera, según Federico:

    - Para un valor cercano a 1, el aprendizaje será más rápido, pero menos seguro, con estimaciones menos estables para los valores Q.
    
    - Para un valor cercano a 0, el aprendizaje será más lento, pero más seguro en llevar una estimación más estable de los valores Q.

- `R(𝑠,a)` = reforzador (reinforcer) o recompensa (reward), después de tomar la acción 𝑎 en el estado 𝑠. Esta recompensa indica el **beneficio obtenido** por realizar esa acción en ese momento específico.

- `γ` = factor de descuento o **discount factor**, también un número entre 0 y 1. Indica cuánto valoramos las recompensas futuras. Un valor cercano a 1 significa que nos importan mucho las recompensas futuras (lo suficiente para considerar las consecuencias a largo plazo de nuestras acciones), mientras que un valor cercano a 0 significa que solo nos interesa la recompensa inmediata.

- `𝑠′` = nuevo estado al que llega el agente después de tomar la acción 𝑎 en el estado 𝑠.

- `max a′ Q(𝑠′, a′)` = selecciona el valor máximo de la función 𝑄 sobre todas las acciones posibles 𝑎′ que el agente podría tomar en el nuevo estado 𝑠′. En otras palabras, el agente trata de estimar el mejor valor futuro posible en el siguiente estado, basándose en lo que ha aprendido hasta el momento.

### Caso de ejemplo

Nos imaginemos un robot que opera en una cuadrícula que representa el salón de una fábrica en la que debe moverse trasladando herramientas.

El objetivo es que el robot aprenda a encontrar el camino más corto, desde su posición inicial hasta el destino, evitando obstáculos.

Entonces, el entorno que vamos a definir para este ejercicio va a ser una cuadrícula de dimensiones de 5x5, un punto de inicio en la esquina superior izquierda, que sería el punto `(0, 0)`, el punto objetivo en la esquina inferior derecha, que sería el punto `(4, 4)` y los obstáculos distribuidos en la cuadrícula.

Vamos a establecer las acciones. Esto implica que el robot va a poder moverse en cuatro direcciones hacia arriba, hacia abajo, izquierda o derecha, y vamos a definir recompensas.

Si alcanza el objetivo, va a tener 100 puntos de premio.

Si colisiona con un obstáculo, se le van a restar 100 puntos y cualquier otro movimiento que haga va a valer -1.

¿Para que?

Para incentivar la eficiencia, que reste lo menos que pueda.

In [1]:
import logging
import random

import numpy as np

In [2]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

console_handler: logging.StreamHandler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)

formatter: logging.Formatter = logging.Formatter(
    '%(message)s'
)

logger.addHandler(console_handler)

In [3]:
grid: tuple[int, int] = (5, 5)

initial_state: tuple[int, int] = (0, 0)
goal_state: tuple[int, int] = (4, 4)

obstacles: list[tuple[int, int]] = [(1, 1), (1, 3), (2, 3), (3, 0)]

up: int = 0
down: int = 1
left: int = 2
right: int = 3

actions: dict[int, tuple[int, int]] = {
    up: (0, -1),
    down: (0, 1),
    left: (-1, 0),
    right: (1, 0),
}

In [4]:
grid_states: int = grid[0] * grid[1]
grid_states

25

In [5]:
possible_actions: int = len(actions)
possible_actions

4

In [6]:
q_table: np.ndarray = np.zeros((grid_states, possible_actions))
q_table

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [8]:
def convert_state_to_index(
        state: tuple[int, int],
        grid: tuple[int, int]
        ) -> int:
    """
    Convert the current two-dimensional representation of the current
    state (the agent's position on the grid) to an unique linear index.

    Every possible state can be represented as a index number for the
    Q table.

    Parameters
    ----------
    state : tuple[int, int]
        The current two-dimensional representation of the current state
        (the agent's position on the grid).

    grid : tuple[int, int]
        The width and height of the grid.

    Returns
    -------
    int
        The index of the current state in the Q table.
    
    Examples
    --------
    >>> convert_state_to_index((0, 0), (5, 5))
    0
    >>> convert_state_to_index((4, 4), (5, 5))
    24

    """
    return state[0] * grid[1] + state[1]

In [9]:
example = convert_state_to_index((1, 0), grid)
example

5

Para entender mejor el resultado de `example`:

![5x5_grid_example.png](./assets/5x5_grid_example.png)

**¿Cómo el agente robot va a aprender a navegar por el entorno?**

Los siguientes parámetros son fundamentales:

In [9]:
alpha: float = 0.1      # learning rate
gamma: float = 0.99     # discount factor
epsilon: float = 0.2    # exploration
episodes: int = 100     # number of episodes

La variable `epsilon` sirve para que el agente no repita siempre las mismas acciones, definiendo la **probabilidad** de que el agente tome una **acción aleatoria** en lugar de la mejor acción conocida hasta el momento, según la tabla Q.

En otras palabras, esto permite que el agente **explore el entorno** desconocido en lugar de explotar constantemente el conocimiento que ya dispone, porque una vez que encuentre una solución, podría tender a repetir esa solución siempre, en vez de explorar otras posibilidades (que podrían ser más eficientes o de mayor interés, según sea el caso).

Un valor alto de `epsilon` (como **0.9**) significa que hay una **alta probabilidad** de que el agente tome una acción aleatoria, promoviendo así la **exploración**.

La variable `episodes` está definiendo el número total de veces (episodios) que se va a repetir este proceso de entrenamiento.

Un episodio comienza con el agente en el estado inicial (declarado en `initial_state`) y termina cuando alcanza el estado objetivo (declarado en `goal_state`) o algún otro criterio de finalización, según sea requerido.

Mientras mayor sea el número de episodios, más oportunidades va a tener el agente de tener más experiencias de las cuales aprender y esto va a mejorar potencialmente sus políticas de acción.

# Q-Learning en Machine Learning - Parte II

In [10]:
def choose_action(
        Q: np.ndarray,
        state: tuple[int, int],
        actions: dict[int, tuple[int, int]],
        grid: tuple[int, int],
        epsilon: float
        ) -> int:
    """
    Choose a random action or the best action for the current state,
    depending on the exploration rate.

    Parameters
    ----------
    Q : np.ndarray
        The Q table.
    
    state : tuple[int, int]
        The current two-dimensional representation of the current state
        (the agent's position on the grid).
    
    actions : dict[int, tuple[int, int]]
        The list of possible actions.

    grid : tuple[int, int]
        The width and height of the grid.

    epsilon : float
        The exploration rate.

    Returns
    -------
    int
        A random action or the best action for the current state,
        depending on the exploration rate.
    """
    random_number: float = random.uniform(0, 1)
    random_action: int = random.choice(list(actions.keys()))
    best_action: int = int(np.argmax(Q[convert_state_to_index(state, grid)]))

    if random_number < epsilon:
        return random_action
    return best_action

In [11]:
def apply_action(
        action: int,
        state: tuple[int, int],
        goal_state: tuple[int, int],
        grid: tuple[int, int],
        obstacles: list[tuple[int, int]]
        ) -> tuple[tuple[int, int], int, bool]:
    """
    Apply the action to the current state.

    Parameters
    ----------
    action : int
        The action to be applied.

    state : tuple[int, int]
        The current two-dimensional representation of the current state
        (the agent's position on the grid).

    goal_state : tuple[int, int]
        The two-dimensional representation of the state declared as
        goal.

    grid : tuple[int, int]
        The width and height of the grid.

    obstacles : list[tuple[int, int]]
        The list of obstacles on the grid.

    Returns
    -------
    tuple[tuple[int, int], int, bool]
        The new state, the reward, and whether the game is over.

    """
    points: int = 0
    is_game_over: bool = False

    new_state: tuple[int, int] = tuple(np.add(state, action) % grid)

    if new_state in obstacles or new_state == state:
        points = -100
        return state, points, is_game_over
    elif new_state == goal_state:
        points = 100
        is_game_over = True
        return new_state, points, is_game_over
    else:
        points = -1
        return new_state, points, is_game_over

In [12]:
def update_Q_values(
        Q: np.ndarray,
        state_index: int,
        action_index: int,
        alpha: float,
        gamma: float,
        new_state_index: int,
        reward: int,
        ) -> float:
    """
    Update the Q values for the current state and action.

    Parameters
    ----------
    Q : np.ndarray
        The Q table.

    state_index : tuple[int, int]
        The index of the current state in the Q table.

    action_index : int
        The index of the current action in the Q table.

    alpha : float
        The learning rate.

    gamma : float
        The discount factor.

    new_state_index : tuple[int, int]
        The index of the new state in the Q table.

    reward : int
        The reward for the current state and action.

    Returns
    -------
    float
        The updated Q value.
    """
    current_q = Q[state_index, action_index]
    best_future_q = np.max(Q[new_state_index])
    return current_q + alpha * (reward + (gamma * best_future_q) - current_q)

In [13]:
for episode in range(episodes):
    logger.info(f'Episode: {episode}')
    state: tuple[int, int] = initial_state
    logger.info(f'State: {state}\n')
    is_game_over: bool = False

    while not is_game_over:
        state_index: int = convert_state_to_index(state, grid)
        logger.info(f'\tState index: {state_index}')
        action_index: int = choose_action(
            q_table,
            state,
            actions,
            grid,
            epsilon
        )
        logger.info(f'\tAction index: {action_index}')
        new_state, reward, is_game_over = apply_action(
            action_index,
            state,
            goal_state,
            grid,
            obstacles
        )
        logger.info(f'\tNew state: {new_state}')
        logger.info(f'\tReward: {reward}')
        logger.info(f'\tIs game over?: {is_game_over}')
        new_state_index: int = convert_state_to_index(new_state, grid)
        logger.info(f'\tNew state index: {new_state_index}')

        q_table[state_index, action_index] = update_Q_values(
            q_table,
            state_index,
            action_index,
            alpha,
            gamma,
            new_state_index,
            reward
        )
        logger.info(f'\tQ value: {q_table[state_index, action_index]}')

        state = new_state
        logger.info(f'\tState: {state}\n')

        if is_game_over:
            logger.info(f'\tEpisode: {episode}, Score: {reward}\n\n')

Episode: 0
State: (0, 0)

	State index: 0
	Action index: 0
	New state: (0, 0)
	Reward: -100
	Is game over?: False
	New state index: 0
	Q value: -10.0
	State: (0, 0)

	State index: 0
	Action index: 1
	New state: (0, 0)
	Reward: -100
	Is game over?: False
	New state index: 0
	Q value: -10.0
	State: (0, 0)

	State index: 0
	Action index: 2
	New state: (2, 2)
	Reward: -1
	Is game over?: False
	New state index: 12
	Q value: -0.1
	State: (2, 2)

	State index: 12
	Action index: 1
	New state: (3, 3)
	Reward: -1
	Is game over?: False
	New state index: 18
	Q value: -0.1
	State: (3, 3)

	State index: 18
	Action index: 0
	New state: (3, 3)
	Reward: -100
	Is game over?: False
	New state index: 18
	Q value: -10.0
	State: (3, 3)

	State index: 18
	Action index: 1
	New state: (4, 4)
	Reward: 100
	Is game over?: True
	New state index: 24
	Q value: 10.0
	State: (4, 4)

	Episode: 0, Score: 100


Episode: 1
State: (0, 0)

	State index: 0
	Action index: 1
	New state: (0, 0)
	Reward: -100
	Is game over?: Fa

### Visualizar el proceso de una manera más gráfica

La **matriz politica** contiene la **mejor acción** que el agente debe tomar **en cada estado** basado en el conocimiento adquirido durante el entrenamiento.

In [14]:
policy_matrix: np.ndarray = np.zeros(grid, dtype=int)
policy_matrix

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [17]:
# Llenar la matriz con la mejor acción posible para cada estado
for i in range(grid[0]):
    for j in range(grid[1]):
        state: tuple[int, int] = (i, j)
        state_index: int = convert_state_to_index(state, grid)
        best_action: int = int(np.argmax(q_table[state_index]))
        policy_matrix[i, j] = best_action

logger.info('### Política aprendida ###\n')
logger.info(policy_matrix)
logger.info('\nLeyenda:')
logger.info('0: arriba\n1: abajo\n2: izquierda\n3: derecha')

### Política aprendida ###

[[3 0 0 0 0]
 [0 0 0 0 0]
 [0 0 2 0 0]
 [0 0 0 1 0]
 [0 0 0 0 0]]

Leyenda:
0: arriba
1: abajo
2: izquierda
3: derecha
